# Today you are a Machine Learning Engineer at Walmart!
For your latest assignemt, your manager has asked your team to work alongside the Data Engineering team to design an optimal Machine Learning pipeline for Automated Inventory, i.e. predicting purchases per customer accurately. The Data Engineering team has used Trifacta to clean some online shopping data gathered from the servers over the last few months. 

You are expected to successfully complete the following tasks:


    Task 1) EDA - find important information about the data
    Task 2) Build a Classifier using Sagemaker


# Load the Data
In this assignment, we will read client data that has been pre-processed (using Trifacta), and stored in an S3 bucket name "mlops-ecommerce". Working off the S3 bucket directly allows you freedom to work from any workstation and also to maintain the integrity of the data sensitivity (no need to download on system).

In [39]:
## Start with loading the Libraries and loading the data
import boto3
import io
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

import sagemaker
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer, json_deserializer

from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

#### For this work, we'll need to load the data provided in the repo!

In [ ]:
role = get_execution_role()
raw_data=pd.read_csv('./preproc_data.csv')

In [3]:
## Importing required Libraries
import os
import numpy as np
import pandas as pd
import seaborn as sb

In [ ]:
#Print the shape of the data
np.shape(raw_data)

In [ ]:
# print first few rows of the data
raw_data.head()

### So you see the Data Engineering team has sent you over 1M records corresponding to users and their interations with products. Notice two things:
1. All features are nuueric (so one-hot-encoding has been done).
2. Last column depicts if the user-product interation resulted in a purchase or not.
Such kind of data is referred to as "User Journeys".

# Task 1: Perform Exploratory Data Analysis (EDA) to find features that are important. 
You can choose any one of (or both of) the two methods given below.

```
Method 1: RandomForestClassifier for feature importance https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html that uses Entropy based measure called GINI Index to rank features of importance

Method 2: Fisher scoring method using the sklearn feature_selection library https://scikit-learn.org/stable/modules/feature_selection.html
```

Observe some differences in the ranked mathods!

In [6]:
#Load Libraries for Data splitting and normalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt
import sklearn.feature_selection as fs
import seaborn as sn

Split the `raw_data` DataFrame into XData (minus the two index columns of `user_id` and `product_id`) and YData

In [7]:
XData = raw_data.iloc[:,2:-1]
YData = raw_data.iloc[:,-1]

Split the data into training and test data, stratifying across YData to ensure equal proportion labels in the training/test sets

In [8]:
X_train, X_test, y_train, y_test = train_test_split(XData,YData,test_size=0.2,random_state=42, stratify=YData)

In [ ]:
X_train.head()

Scale the data

In [10]:
MMscaler = MinMaxScaler()
X_train = MMscaler.fit_transform(X_train)
X_test = MMscaler.transform(X_test)

Initialize, and fit an estimator for use in finding feature importance

In [11]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 10))

In [ ]:
sel.fit(X_train, y_train)

In [13]:
selected_feat= XData.columns[(sel.get_support())]

In [ ]:
importances = sel.estimator_.feature_importances_
importances

In [ ]:
indices = np.argsort(importances)[::-1] 
colname = XData.columns[indices]
plt.figure(figsize=(15,9))
plt.title("Feature importances",size=20)
sn.barplot(x=colname, y=importances[indices],palette="deep")
plt.xticks(rotation=90,size=20)
plt.show()

In [ ]:
kb = fs.SelectKBest(k=X_train.shape[1])
kb.fit(X_train, y_train)
names = XData.columns.values[kb.get_support()]
scores = kb.scores_[kb.get_support()]
names_scores = list(zip(names, scores))

In [17]:
fScoreDF = pd.DataFrame(data = names_scores, columns=['Feat_names','F_Scores'])
fScoreDF_sorted = fScoreDF.sort_values(['F_Scores','Feat_names'], ascending =[False, True])

In [ ]:
plt.figure(figsize=(15,9))
sn.barplot(x= "Feat_names", y="F_Scores",data=fScoreDF_sorted)
plt.xticks(rotation=90,size=20)
plt.show()

## Thus, we observe that the temporal features (time, day, year etc) have significantly less importance than the other features. Thus, we reduce data dimensionality by discarding the temporal features!

In [19]:
X_train_1=X_train[:,1:13]
X_test_1=X_test[:,1:13]

In [ ]:
# Next lets look at the data distribution
plt.hist(y_train)
plt.hist(y_test)
plt.show()
print("Fraction of Purchases in train data=", np.sum(y_train)/np.shape(y_train)[0])
print("Fraction of Purchases in test data=", np.sum(y_test)/np.shape(y_test)[0])

# Task 2 [Instructor Led]: Build a classifier hyper-parameterization pipeline with Sagemaker for complete dataset.

## Step 1:  Here, we convert the processed training data to protobuf and write to S3 for linear-learner in Sagemaker. Ensure the "New bucket name" is accurate in the cell underneath.

In [22]:
bucket_new = None # replace None with <FIRSTNAME>mlops
prefix = None # replace this with your own prefix
s3_train_key = "{}/train/recordio-pb-data".format(prefix)
s3_train_path = os.path.join("s3://", bucket_new, s3_train_key)
vectors = np.array([t.tolist() for t in X_train_1]).astype("float32")
labels = np.array(y_train).astype("float32")
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, vectors, labels)
buf.seek(0)
boto3.resource("s3").Bucket(bucket_new).Object(s3_train_key).upload_fileobj(buf)


We will wrap the model training setup in a convenience function that takes in the S3 location of the training data, the model hyperparameters that define our training job, and the S3 output path for model artifacts. Inside the function, we'll hardcode the algorithm container, the number and type of EC2 instances to train on, and the input and output data formats.

In [44]:
from sagemaker.image_uris import retrieve as retrieve_image_uris

def predictor_from_hyperparams(s3_train_data, hyperparams, output_path):
    """
    Create an Estimator from the given hyperparams, fit to training data, and return a deployed predictor
    """
    # specify algorithm containers and instantiate an Estimator with given hyperparams
    container = retrieve_image_uris("linear-learner", boto3.Session().region_name)

    linear = sagemaker.estimator.Estimator(
        container,
        role,
        instance_count=1,
        instance_type="ml.m4.xlarge",
        output_path=output_path,
        sagemaker_session=sagemaker.Session(),
    )
    linear.set_hyperparameters(**hyperparams)
    # train model
    linear.fit({"train": s3_train_data})
    # deploy a predictor
    linear_predictor = linear.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")
    #linear_predictor.content_type = "csv"
    linear_predictor.serializer = CSVSerializer()
    linear_predictor.deserializer = JSONDeserializer()
    return linear_predictor

And add another convenience function for setting up a hosting endpoint, making predictions, and evaluating the model. To make predictions, we need to set up a model hosting endpoint. Then we feed test features to the endpoint and receive predicted test labels. To evaluate the models we create in this exercise, we'll capture predicted test labels and compare them to actuals using some common binary classification metrics.

In [32]:
def evaluate(linear_predictor, test_features, test_labels, model_name, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  Return binary classification metrics.
    """
    # split the test data set into 100 batches and evaluate using prediction endpoint
    prediction_batches = [
        linear_predictor.predict(batch)["predictions"]
        for batch in np.array_split(test_features, 100)
    ]
    # parse raw predictions json to exctract predicted label
    test_preds = np.concatenate(
        [np.array([x["predicted_label"] for x in batch]) for batch in prediction_batches]
    )

    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1 - test_labels, test_preds).sum()
    tn = np.logical_and(1 - test_labels, 1 - test_preds).sum()
    fn = np.logical_and(test_labels, 1 - test_preds).sum()

    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    f1 = 2 * precision * recall / (precision + recall)

    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=["actuals"], colnames=["predictions"]))
        print("\n{:<11} {:.3f}".format("Recall:", recall))
        print("{:<11} {:.3f}".format("Precision:", precision))
        print("{:<11} {:.3f}".format("Accuracy:", accuracy))
        print("{:<11} {:.3f}".format("F1:", f1))

    return {
        "TP": tp,
        "FP": fp,
        "FN": fn,
        "TN": tn,
        "Precision": precision,
        "Recall": recall,
        "Accuracy": accuracy,
        "F1": f1,
        "Model": model_name,
    }

And finally we'll add a convenience function to delete prediction endpoints after we're done with them:

In [33]:
def delete_endpoint(predictor):
    try:
        boto3.client("sagemaker").delete_endpoint(EndpointName=predictor.endpoint)
        print("Deleted {}".format(predictor.endpoint))
    except:
        print("Already deleted: {}".format(predictor.endpoint))

Let's begin by training a binary classifier model with the linear learner default settings. Note that we're setting the number of epochs to 10. Also, notice that the the function above has an EARLY STOPPING CRITERIA set to when Recall reaches 0.7. See more on Early Stopping Criteria in https://github.com/aws/amazon-sagemaker-examples/blob/master/scientific_details_of_algorithms/linear_learner_class_weights_loss_functions/linear_learner_class_weights_loss_functions.ipynb

With early stopping, we don't have to worry about setting the number of epochs too high. Linear learner will stop training automatically after the model has converged.

## Step 2: Initiate hyperpramatereization using the pipeline below.
### For each Breakout group members should consider using either one of the celles below (labelled OPTION 1, OPTION 2) and then sharing results to see which are the best. Running both can take a LONG time. So for timely completion please selection either OPTION 1 or 2.

## Option 1

In [ ]:
# [OPTION 1]: Training a binary classifier (Logistic Regression) with automated threshold tuning

autothresh_hyperparams = {
    "feature_dim": 12,
    "predictor_type": "binary_classifier",
    "binary_classifier_model_selection_criteria": "precision_at_target_recall",
    "target_recall": 0.8,
    "epochs": 10,
}
autothresh_output_path = "s3://{}/{}/autothresh/output".format(bucket_new, prefix)
autothresh_predictor = predictor_from_hyperparams(
    s3_train_path, autothresh_hyperparams, autothresh_output_path
)

In [ ]:
# Evaluation for OPTION 1
predictors = {
    "Logistic with auto threshold": autothresh_predictor,
         
}
metrics = {
    key: evaluate(predictor, X_test_1, y_test, key, False)
    for key, predictor in predictors.items()
}
pd.set_option("display.float_format", lambda x: "%.3f" % x)
display(
    pd.DataFrame(list(metrics.values())).loc[:, ["Model", "Recall", "Precision", "Accuracy", "F1"]]
)

## Option 2

In [ ]:
# [OPTION 2]: Training a binary classifier with hinge loss, balanced class weights, and automated threshold tuning
svm_balanced_hyperparams = {
    "feature_dim": 12,
    "predictor_type": "binary_classifier",
    "loss": "hinge_loss",
    "binary_classifier_model_selection_criteria": "precision_at_target_recall",
    "target_recall": 0.8,
    "positive_example_weight_mult": "balanced",
    "epochs": 10,
}
svm_balanced_output_path = "s3://{}/{}/svm_balanced/output".format(bucket_new, prefix)
svm_balanced_predictor = predictor_from_hyperparams(
    s3_train_path, svm_balanced_hyperparams, svm_balanced_output_path
)

In [ ]:
# Evaluation for OPTION 2
predictors = {
    "Hinge with class weights": svm_balanced_predictor,
    
}
metrics = {
    key: evaluate(predictor, X_test_1, y_test, key, False)
    for key, predictor in predictors.items()
}
pd.set_option("display.float_format", lambda x: "%.3f" % x)
display(
    pd.DataFrame(list(metrics.values())).loc[:, ["Model", "Recall", "Precision", "Accuracy", "F1"]]
)

## Clean Up

In [ ]:
#Finally, clean up all the predictors
for predictor in [
    autothresh_predictor,
    svm_balanced_predictor,
]:
    delete_endpoint(predictor)